# Random Clique Cover for as Graph to (Latent) Simplicial Complex Lifting
Large real-world graphs tend to be **sparse**, but they often contain **many densely connected subgraphs** and exhibit high clustering coefficients.  

***

Here we hinge on the recently proposed Bayesian nonparametric random graph model from [1] for **random clique covers**. The resulting distribution is guaranteed to capture power law degree distribution, global sparsity and non-vanishing local clustering coefficient.

In the original work [1], the distribution has been used as a prior on observed graphs. In particular, in the Bayesian setting, given a graph, the model is useful to obtain a distribution on latent **clique complexes**, i.e. a specific class of **simplicial complexes**, whose 1-skeleton structural properties are consistent with the ones of the graph used to compute the likelihood.

***

In the context of **Topological Deep Learning** [2][3] and the very recently emerged paradigm of **Latent Topology Inference** (LTI) [4], it is natural to look at the model in [1] as a novel LTI method able to infer a **random latent simplicial complex** from an input graph. Or, in other words, to use [1] as a novel random lifting procedure from graphs to simplicial complexes.

Next, we provide a quick introduction to the model in [1]. For a more in-depth exposition, please refer to the paper. To the best of our knowledge, this is the first random lifting procedure relying on Bayesian arguments.

To summarize, this is:
- a **non-deterministic** lifting,
- **not present** in the literature as a lifting prcedure,
- **based on connectivity**,
-  **modifying** the initial connectivity of the graph.

# The Random Clique Cover Model
[MAURICIO]


### Imports and utilities

In [1]:
# With this cell any imported module is reloaded before each cell execution
%load_ext autoreload
%autoreload 2
from modules.data.load.loaders import GraphLoader
from modules.data.preprocess.preprocessor import PreProcessor
from modules.utils.utils import (
    describe_data,
    load_dataset_config,
    load_model_config,
    load_transform_config,
)

## Loading the Dataset

Here we just need to spicify the name of the available dataset that we want to load. First, the dataset config is read from the corresponding yaml file (located at `/configs/datasets/` directory), and then the data is loaded via the implemented `Loaders`.


In [2]:
dataset_name = "cocitation_cora"
dataset_config = load_dataset_config(dataset_name)
loader = GraphLoader(dataset_config)


Dataset configuration for cocitation_cora:

{'data_domain': 'graph',
 'data_type': 'cocitation',
 'data_name': 'Cora',
 'data_dir': 'datasets/graph/cocitation',
 'num_features': 1433,
 'num_classes': 7,
 'task': 'classification',
 'loss_type': 'cross_entropy',
 'monitor_metric': 'accuracy',
 'task_level': 'node'}


We can then access to the data through the `load()`method:

In [3]:
dataset = loader.load()
describe_data(dataset)


Dataset only contains 1 sample:
 - Graph with 2708 vertices and 10556 edges.
 - Features dimensions: [1433, 0]
 - There are 0 isolated nodes.



## Loading and Applying the Lifting

In this section we will instantiate the random clique lifting.

For simplicial complexes creating a lifting involves creating a `SimplicialComplex` object from topomodelx and adding simplices to it using the method `add_simplices_from`. The `SimplicialComplex` class then takes care of creating all the needed matrices.

Similarly to before, we can specify the transformation we want to apply through its type and id --the correxponding config files located at `/configs/transforms`.

Note that the *tranform_config* dictionary generated below can contain a sequence of tranforms if it is needed.

In [4]:
# Define transformation type and id
transform_type = "liftings"
# If the transform is a topological lifting, it should include both the type of the lifting and the identifier
transform_id = "graph2simplicial/latent_clique_cover_lifting"

# Read yaml file
transform_config = {
    "lifting": load_transform_config(transform_type, transform_id)
    # other transforms (e.g. data manipulations, feature liftings) can be added here
}


Transform configuration for graph2simplicial/latent_clique_cover_lifting:

{'transform_type': 'lifting',
 'transform_name': 'LatentCliqueCoverLifting',
 'complex_dim': 3,
 'preserve_edge_attr': False,
 'signed': True,
 'feature_lifting': 'ProjectionSum'}


We than apply the transform via our `PreProcesor`:

In [5]:
lifted_dataset = PreProcessor(dataset, transform_config, loader.data_dir)
describe_data(lifted_dataset)

Processing...


## Create and Run a Simplicial NN Model

In this section a simple model is created to test that the used lifting works as intended. In this case the model uses the `up_laplacian_1` and the `down_laplacian_1` so the lifting should make sure to add them to the data.

In [ ]:
from modules.models.simplicial.san import SANModel

model_type = "simplicial"
model_id = "san"
model_config = load_model_config(model_type, model_id)

model = SANModel(model_config, dataset_config)


Model configuration for simplicial SAN:

{'in_channels': None,
 'hidden_channels': 32,
 'out_channels': None,
 'n_layers': 2,
 'n_filters': 2,
 'order_harmonic': 5,
 'epsilon_harmonic': 0.1}


In [ ]:
y_hat = model(lifted_dataset.get(0))

If everything is correct the cell above should execute without errors.

# References

***
[[1]](http://proceedings.mlr.press/v115/williamson20a/williamson20a.pdf) Williamson, Sinead A., and Mauricio Tec. "Random clique covers for graphs with local density and global sparsity." Uncertainty in Artificial Intelligence (UAI). PMLR, 2020.

[[2]](https://arxiv.org/abs/2402.08871)  Papamarkou, Theodore, et al. "Position paper: Challenges and opportunities in topological deep learning." arXiv preprint arXiv:2402.08871 (2024).

[[3]](https://arxiv.org/abs/2206.00606) Hajij, Mustafa, et al. "Topological deep learning: Going beyond graph data." arXiv preprint arXiv:2206.00606 (2022).

[[4]](https://openreview.net/forum?id=0JsRZEGZ7L) Battiloro, Claudio, et al. "From latent graph to latent topology inference: Differentiable cell complex module." The Twelfth International Conference on Learning Representations (ICLR), 2024.

***
